# MoBioFP - Model Training For Fingertip Semantic Segmentation

## Import Python libraries

In [ ]:
import os
import cv2
import imutils
import platform
import albumentations as A
import matplotlib.pyplot as plt

from keras import backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.optimizers.legacy import Adam as AdamLegacy
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from mobiofp.segmentation import Segment, DataGenerator

## Global constants

In [ ]:
IMAGE_DIR_PATH = "UNET_DATASET_IMAGE_DIR_PATH"
LABELS_DIR_PATH = "UNET_DATASET_MASK_DIR_PATH"
MODEL_CHECKPOINT_PATH = "UNET_MODEL_CHECKPOINT_PATH"
EPOCHS = 100

## Define data augmentation functions

In [ ]:
def round_clip_0_1(x, **kwargs):
    """
    Rounds the input to the nearest integer and clips it to the range [0, 1].

    Parameters:
        x (np.array): The input array to round and clip.
        **kwargs: Arbitrary keyword arguments. This is included to maintain compatibility with the albumentations library, which may pass additional arguments.

    Returns:
        np.array: The rounded and clipped input array.
    """
    return x.round().clip(0, 1)


def training_augmentation():
    """
    Defines the augmentation pipeline for training data.

    Returns:
        albumentations.Compose: The augmentation pipeline.
    """
    train_transform = [
        A.PadIfNeeded(min_height=256, min_width=256, always_apply=True, border_mode=0),
        # Flip augmentations
        A.OneOf(
            [A.HorizontalFlip(p=1), A.VerticalFlip(p=1), A.Transpose(p=1)],
            p=0.9,
        ),
        # Geometric augmentations
        A.OneOf(
            [
                A.ShiftScaleRotate(
                    scale_limit=0.3,
                    rotate_limit=45,
                    shift_limit=0.2,
                    border_mode=0,
                    p=1,
                ),
                A.Perspective(p=1),
            ],
            p=0.9,
        ),
        # Resolution augmentation
        A.OneOf(
            [
                A.Sharpen(p=1),
                A.Blur(blur_limit=3, p=1),
                A.MotionBlur(blur_limit=3, p=1),
            ],
            p=0.9,
        ),
        # Visual alterations
        A.OneOf(
            [
                A.HueSaturationValue(
                    hue_shift_limit=1, sat_shift_limit=0.2, val_shift_limit=0.5, p=1
                ),
                A.RandomBrightnessContrast(p=1),
            ],
            p=0.5,
        ),
        A.Lambda(mask=round_clip_0_1),
    ]

    return A.Compose(train_transform)


def validation_augmentation():
    """
    Defines the augmentation pipeline for validation data.

    Returns:
        albumentations.Compose: The augmentation pipeline.
    """
    test_transform = [A.PadIfNeeded(256, 256)]

    return A.Compose(test_transform)

## Create dataset

Load images and semantic segmentation labels, dividing the dataset into a training subset (85%) and a validation subset (15%).

In [ ]:
images_dir = sorted(os.listdir(IMAGE_DIR_PATH))
labels_dir = sorted(os.listdir(LABELS_DIR_PATH))
training_images, validation_images, training_labels, validation_labels = train_test_split(
    images_dir, labels_dir, test_size=0.15, random_state=42
)

# Check if the dataset is loaded correctly
assert len(training_images) == len(training_labels) and len(validation_images) == len(
    validation_labels
), "Dataset not loaded correctly"

In [ ]:
# Define training and validation parameters
training_params = {
    "augmentation": training_augmentation(),
    "preprocessing": None,
    "batch_size": 8,
    "dim": (256, 256, 3),
    "shuffle": True,
}
validation_params = {
    "augmentation": validation_augmentation(),
    "preprocessing": None,
    "batch_size": 8,
    "dim": (256, 256, 3),
    "shuffle": True,
}

# Generate training and validation datasets using the DataGenerator class
training_dataset = DataGenerator(
    training_images, IMAGE_DIR_PATH, training_labels, LABELS_DIR_PATH, **training_params
)
validation_dataset = DataGenerator(
    validation_images, IMAGE_DIR_PATH, validation_labels, LABELS_DIR_PATH, **validation_params
)

## Define model metrics functions

In [ ]:
def jaccard_distance_loss(y_true, y_pred, smooth=100):
    """
    Calculates the Jaccard distance loss between the true and predicted labels.

    Parameters:
    y_true (tf.Tensor): The true labels.
    y_pred (tf.Tensor): The predicted labels.
    smooth (int, optional): A smoothing factor to prevent division by zero. Defaults to 100.

    Returns:
        tf.Tensor: The Jaccard distance loss.
    """
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    sum_ = K.sum(K.abs(y_true) + K.abs(y_pred), axis=-1)
    jac = (intersection + smooth) / (sum_ - intersection + smooth)

    return (1 - jac) * smooth


def dice_coef(y_true, y_pred):
    """
    Calculates the Dice coefficient between the true and predicted labels.

    Parameters:
        y_true (tf.Tensor): The true labels.
        y_pred (tf.Tensor): The predicted labels.

    Returns:
        tf.Tensor: The Dice coefficient.
    """
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)

    return (2.0 * intersection + K.epsilon()) / (K.sum(y_true_f) + K.sum(y_pred_f) + K.epsilon())

## Define the model

In [ ]:
model = Segment()
model.info()

if platform.system() == "Darwin":
    # Use the legacy Adam optimizer on M1/M2 Macs
    optim = AdamLegacy(learning_rate=0.0001)
else:
    # Use the new Adam optimizer on other platforms
    optim = Adam(learning_rate=0.0001)

model.compile(
    optimizer=optim,
    loss=jaccard_distance_loss,
    metrics=[
        dice_coef,
        "accuracy",
    ],
)

## Train the model

In [ ]:
loss0, dice_coef0, accuracy0 = model.evaluate(validation_dataset)

In [ ]:
print(f"initial loss: {loss0:.2f}")
print(f"initial dice coefficient: {dice_coef0:.2f}")
print(f"initial accuracy: {accuracy0:.2f}")

In [ ]:
callbacks = [
    ModelCheckpoint(
        MODEL_CHECKPOINT_PATH,
        save_weights_only=True,
        save_best_only=True,
        mode="min",
    ),
    ReduceLROnPlateau(),
    EarlyStopping(mode="max", monitor="val_dice_coef", patience=3, verbose=1),
]
history = model.train(training_dataset, validation_dataset, epochs=EPOCHS, callbacks=callbacks)

## Learning curves

In [ ]:
acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]

loss = history.history["loss"]
val_loss = history.history["val_loss"]

dice_coef = history.history["dice_coef"]
val_dice_coef = history.history["val_dice_coef"]

plt.figure(figsize=(10, 10))
plt.subplot(3, 1, 1)
plt.plot(acc, label="Training Accuracy")
plt.plot(val_acc, label="Validation Accuracy")
plt.legend(loc="lower right")
plt.ylim([min(plt.ylim()), 1])
plt.title("Training and Validation Accuracy")

plt.subplot(3, 1, 2)
plt.plot(loss, label="Training Loss")
plt.plot(val_loss, label="Validation Loss")
plt.legend(loc="upper right")
plt.ylim([0, 1.0])
plt.title("Training and Validation Loss")

plt.subplot(3, 1, 3)
plt.plot(dice_coef, label="Training Dice Coefficient")
plt.plot(val_dice_coef, label="Validation Dice Coefficient")
plt.legend(loc="upper left")
plt.ylim([0, 1.0])
plt.title("Training and Validation Dice Coefficient")

plt.tight_layout()
plt.show()

## Make predictions

In [ ]:
image = cv2.imread("../data/raw/samples/1_i_1_w_1.jpg")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image = imutils.rotate_bound(image, 90)
result = model.predict(image)

plt.figure(figsize=(10, 10))
plt.subplot(1, 2, 1)
plt.imshow(image)
plt.title("Original Image")
plt.axis("off")
plt.subplot(1, 2, 2)
plt.imshow(result, cmap="gray")
plt.title("Segmented Image")
plt.axis("off")
plt.tight_layout()
plt.show()